# Module 3.2 Add-on: `circuit.parameters` shows only unassigned symbols  

In [1]:

# Metadata
import sys, platform
from importlib.metadata import version, PackageNotFoundError
def safe_ver(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return "not installed"
print("Python:", sys.version.split()[0])
print("Qiskit:", safe_ver("qiskit"))
print("qiskit-ibm-runtime:", safe_ver("qiskit-ibm-runtime"))


Python: 3.12.9
Qiskit: 2.2.0
qiskit-ibm-runtime: 0.41.1


In [2]:

# Imports (Qiskit 2.x APIs)
from math import pi
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2, EstimatorV2
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

backend = FakeSherbrooke()



## Conceptual Overview  

`Parameter` is a scalar symbol; `ParameterVector` is an ordered collection of  
symbols. `assign_parameters` binds symbolic parameters to numeric values (or to  
other symbols). It accepts dictionaries (including vector-as-key), or a sequence  
aligned to the circuit's parameter order. The `inplace` flag chooses whether to  
modify the circuit or return a new one.  

Always compile the bound circuit for the target backend (here a fake backend)  
before running `SamplerV2`/`EstimatorV2`.  


### 3.2.0  Create and bind a single `Parameter`  

**Docs:**  
- Parameter:  
https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.Parameter  

Notice parameters may be used more than once, and not all parameters need to be used

In [3]:
from qiskit.circuit import Parameter
from math import pi
phi = Parameter("φ")
g = Parameter("g")
qc = QuantumCircuit(1)
qc.rx(phi, 0)
qc.ry(phi/2, 0)
qc.measure_all()
qc.draw()

┌───────┐┌─────────┐ ░ ┌─┐
     q: ┤ Rx(φ) ├┤ Ry(φ/2) ├─░─┤M├
        └───────┘└─────────┘ ░ └╥┘
meas: 1/════════════════════════╩═
                                0


### 3.2.1  Build structured families with `ParameterVector`  

**Docs:**  
- ParameterVector:  
  https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.ParameterVector
- ParameterVector can be used to assign vectors of values as parameters.  It's a more convenient way to manage many parameters at once.

In [4]:

vec = ParameterVector("θ", 2)
qc = QuantumCircuit(1)
qc.ry(vec[0], 0)
qc.rz(vec[1], 0)
qc.measure_all()

assign_map = {vec[0]: 0.2, vec[1]: 0.1}
bqc = qc.assign_parameters(assign_map)
tqc = transpile(bqc, backend)
sampler = SamplerV2(mode=backend)
job = sampler.run([tqc], shots=512)
results = job.result()
print(results[0].data.meas.get_counts())


{'0': 500, '1': 12}



## 3.2.3 Assigning Parameters

- We create a circuit using only a subset of symbols from a `ParameterVector`.  
`circuit.parameters` lists the **unassigned** symbols still present. After  
`assign_parameters(..., inplace=True)`, the set becomes empty if all used  
symbols were bound.
- We will also show that you cannot assign parameters that are not used in the circuit.
- There are three ways to assign parameters via `assign_parameters` method:
1)  `assign_parameters` with a dictionary (scalar keys)
2)  `assign_parameters` using a `ParameterVector` as the key
3)  `assign_parameters` with a positional sequence  (same order as they were created)


### 3.2.3.1  `assign_parameters` with a dictionary (scalar keys)  

Map each `Parameter` to a numeric value. This is explicit and common.  

In [5]:

θ, ϕ = Parameter("θ"), Parameter("φ")
qc = QuantumCircuit(1)
qc.rx(θ, 0)
qc.rz(ϕ, 0)
qc.measure_all()

mapped = qc.assign_parameters({θ: 0.3, ϕ: 0.15})
print("params now:", sorted(p.name for p in mapped.parameters))



params now: []



### 3.2.3.2  `assign_parameters` using a `ParameterVector` as the key  

In [6]:

thetas = ParameterVector("θ", 3)
qc = QuantumCircuit(1)
qc.rx(thetas[0], 0)
qc.ry(thetas[1], 0)
qc.rz(thetas[2], 0)
qc.measure_all()

mapped = qc.assign_parameters({thetas: [0.1, 0.2, 0.3]})
print("Remaining unbound params:", sorted(p.name for p in mapped.parameters))

Remaining unbound params: []



### 3.2.3.3  `assign_parameters` with a positional sequence  

In [7]:

a, b = Parameter("a"), Parameter("b")
qc = QuantumCircuit(1)
qc.ry(a, 0)
qc.rz(b, 0)
qc.measure_all()

print("Parameter order:", [p.name for p in qc.parameters])
mapped = qc.assign_parameters([0.25, 0.5])  # a -> 0.25, b -> 0.5
print("After assign:", sorted(p.name for p in mapped.parameters))

Parameter order: ['a', 'b']
After assign: []


## 3.2.4  `inplace` argument: return-new vs modify-in-place  

`inplace` keyword may be used to return a copy of the circuit  
if set to `False`.  If set to `True`, then nothing is returned   
and the circuit is overwritten (modified in place).

In [8]:

theta = ParameterVector('theta', 3)  # theta[0], theta[1], theta[2]
phi = ParameterVector('phi', 2)      # not used here

cir = QuantumCircuit(2, 2)
cir.h(0)
cir.rx(theta[0], 0)   # uses theta[0]
cir.ry(theta[1], 1)   # uses theta[1]

print("Before assignment, cir.parameters should show theta[0] and theta[1]:")
print(sorted(p.name for p in cir.parameters))

# ---- Uncomment to show error ----
# Try to assign all entries of 'theta' using vector-as-key mapping 
# cir.assign_parameters({theta: [0.1, 0.1, 0.1]}, inplace=True)  

cir.assign_parameters({theta[0]: 0.1, theta[1]:0.1}, inplace=True)  

print("\nAfter in-place assignment, cir.parameters should be empty:")
print(sorted(p.name for p in cir.parameters))


Before assignment, cir.parameters should show theta[0] and theta[1]:
['theta[0]', 'theta[1]']

After in-place assignment, cir.parameters should be empty:
[]


Notice that even though we did not use `theta[2]` it doesn't appear when we print   
`cir.parameters` because only parameters that are used in the circuit are printed.

### **Common pitfalls**

- The `ParameterVector` constructor takes as input in this order:
  `ParameterVector(<name>, <size>)`.
- This is in contrast to `QuantumRegister` which takes as input:
  `QuantumRegister(<size>, <name)`.


## 3.2.5 ParameterExpression
- A ParameterExpression is a symbolic algebraic expression built  
from one or more Parameters (e.g., 2*theta + phi) that you can  
place anywhere a numeric gate parameter would go.

- It lets you encode relationships/constraints between angles  
(shared values, linear combos, offsets), so binding a few base  
parameters consistently updates all dependent places in the circuit.

- You can partially bind it (remaining symbols stay symbolic) or fully  
bind it to a number; this supports workflows like template circuits,
 variational forms, and parameter tying across layers.

**Docs:**  
- ParameterExpression:  
https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.ParameterExpression  

A `ParameterExpression` represents symbolic algebra over parameters, e.g.  
$\alpha = 2\theta + \phi$. Bind the constituent parameters to evaluate the  
expression numerically.  

In [9]:

theta = Parameter("θ"); phi = Parameter("φ")
alpha = 2*theta + phi
beta  = theta/2 - phi
qc = QuantumCircuit(1)
qc.rx(alpha, 0); qc.rz(beta, 0); qc.measure_all()
print("Unbound:", sorted(p.name for p in qc.parameters))
pqc = qc.assign_parameters({theta: 0.4})
print("After partial bind:", sorted(p.name for p in pqc.parameters))
fbqc = qc.assign_parameters({theta: 0.4, phi: 0.1})
print("After full bind:", sorted(p.name for p in fbqc.parameters))


Unbound: ['θ', 'φ']
After partial bind: ['φ']
After full bind: []



### Example 1 — `.assign`: replace one parameter with a number or expression  

In [11]:

from qiskit.circuit import Parameter

# α = 2*θ + φ
theta, phi = Parameter("θ"), Parameter("φ")
alpha = 2*theta + phi

print("alpha parameters:", sorted(p.name for p in alpha.parameters))

# Replace θ with a numeric constant
alpha_num = alpha.assign(theta, 0.3)
print("\nalpha.assign(theta, 0.3):", alpha_num)
print("remaining params:", sorted(p.name for p in alpha_num.parameters))

# Replace φ with another expression, e.g., φ -> (θ/2)
alpha_expr = alpha.assign(phi, theta/2)
print("\nalpha.assign(phi, theta/2):", alpha_expr)
print("remaining params:", sorted(p.name for p in alpha_expr.parameters))

# You can chain assigns
alpha_chain = alpha.assign(theta, 0.5).assign(phi, 0.1)
print("\nalpha.assign(...).assign(...):", alpha_chain)
try:
    print("numeric via float(...):", float(alpha_chain))
except Exception as e:
    print("could not cast to float:", e)


alpha parameters: ['θ', 'φ']

alpha.assign(theta, 0.3): 0.6 + φ
remaining params: ['φ']

alpha.assign(phi, theta/2): 2.5*θ
remaining params: ['θ']

alpha.assign(...).assign(...): 1.1
numeric via float(...): 1.1



### Example 2 — `.subs`: replace parameters by other expressions (possibly  symbolic)  

In [12]:

# β = 3*θ - 2*φ
theta, phi = Parameter("θ"), Parameter("φ")
beta = 3*theta - 2*phi
print("beta parameters:", sorted(p.name for p in beta.parameters))

# Map θ -> (φ + 0.2), φ -> (θ - 0.1). This swaps symbols with offset, still symbolic.
beta_swapped = beta.subs({theta: phi + 0.2, phi: theta - 0.1})
print("\nbeta.subs({θ: φ+0.2, φ: θ-0.1}):", beta_swapped)
print("remaining params:", sorted(p.name for p in beta_swapped.parameters))

# Now fully bind via .bind on the resulting expression
beta_bound = beta_swapped.bind({theta: 0.4, phi: 0.1})
print("\nafter bind mapping {θ: 0.4, φ: 0.1} ->", beta_bound)
try:
    print("numeric via float(...):", float(beta_bound))
except Exception as e:
    print("could not cast to float:", e)


beta parameters: ['θ', 'φ']

beta.subs({θ: φ+0.2, φ: θ-0.1}): 3*(0.2 + φ) + (-2)*(-0.1 + θ)
remaining params: ['θ', 'φ']

after bind mapping {θ: 0.4, φ: 0.1} -> 0.30000000000000004
numeric via float(...): 0.30000000000000004



### Example 3 — Use assigned/substituted expressions inside a circuit  

In [14]:

from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_ibm_runtime import SamplerV2

backend = FakeSherbrooke()

θ, φ = Parameter("θ"), Parameter("φ")
expr = 2*θ + φ
expr = expr.assign(θ, φ/2)        # 2*(φ/2) + φ = 2*φ
expr = expr.subs({φ: θ + 0.1})    # 2*(θ+0.1) = 2θ + 0.2

qc = QuantumCircuit(1)
qc.ry(expr, 0)
qc.measure_all()

# Bind θ and run
bqc = qc.assign_parameters({θ: 0.3})
tqc = transpile(bqc, backend)
res = SamplerV2(mode=backend).run([tqc], shots=256).result()


## 3.2.7 Basic Multiple Choice Questions  

**Q1.** Which call creates `θ[0]`, `θ[1]`, `θ[2]`?  
A. `[Parameter('θ')]*3`  
B. `ParameterVector('θ', 3)`  
C. `Parameter('θ', 3)`  
D. `ParameterVector(3, 'θ')`  

**Q2.** What does `inplace=True` do for `assign_parameters`?  
A. Returns a new circuit.  
B. Modifies the circuit and typically returns `None`.  
C. Throws unless all parameters are bound.  
D. Has no effect.  

**Q3.** Which mapping binds all entries of a vector at once?  
A. `{θ: [0.1, 0.2, 0.3]}` where `θ = ParameterVector('θ', 3)`  
B. `{θ[0]: [0.1, 0.2, 0.3]}`  
C. `{θ: 0.1}`  
D. `[0.1, 0.2, 0.3, 0.4]`  

<details>  
<summary><b>Answer Key</b></summary>  

Q1: B  
Q2: B  
Q3: A  

</details>  

## 3.2.8 Advanced Multiple Choice Questions

**Q1.** Given the code snippet below, what is the most likely output?
```python
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

th  = ParameterVector('th', 3)    # th[0], th[1], th[2]
phi = Parameter('phi')

qc = QuantumCircuit(1)
qc.rx(th[2], 0)
qc.rz(phi, 0)
qc.ry(th[0], 0)

vals = [0.1, 0.2, 0.3]
bound = qc.assign_parameters(vals)
print([op.operation.params[0] for op in bound.data])  # gate parameters in gate order
```
A. [0.3, 0.1, 0.2]

B. [0.1, 0.2, 0.3]

C. [0.2, 0.3, 0.1]

D. Raises an exception because vals is not a dict

**Q3**. Which statement about `assign_parameters` is correct?

A. When passed a list/array, values are bound in the order gates  
were added to the circuit.  

B. With `inplace=True`, the circuit is modified and   
the method typically returns None.  

C. A mapping with a `ParameterVector` key cannot be   
used unless the vector appears in every gate.  

D. Using `flat_input=True` allows string keys like  
'theta' or 'theta[0]' in the mapping.

<details>
<summary><b>Answer Key with Brief Explanations</b></summary>

**Q1: A**

* **Why A:** Lists bind by parameter order (not gate order); mapping is `phi→0.1`, `th[0]→0.2`, `th[2]→0.3`, so gate-order print is `[0.3, 0.1, 0.2]`.
* **Not B/C:** Assume binding follows gate order or wrong mapping.
* **Not D:** Lists are valid for `assign_parameters`.

**Q2: A**

* **Why A:** `expr.parameters` contains `x`, `t[0]`, `t[1]`; sorting their names yields that list.
* **Not B:** Vectors contribute elements, not a single `'t'`.
* **Not C/D:** Parameters are present; not just `x` and not empty.

**Q3: B**

* **Why B:** `inplace=True` mutates the circuit and usually returns `None`.  Modifies the circuit inplace.
* **Not A:** List binding uses parameter order, not gate order.
* **Not C:** You can bind a whole vector with one mapping entry.
* **Not D:** `flat_input=True` requires `Parameter` keys, not strings.

</details>


</details> 